## Data Pre-processing

### What are the positive % of training data? What about the dev set? Does it make sense given your knowledge of the average per capita income in the US?

In [28]:
# Training data
# Target categories and their counts
!cat hw1-data/income.train.txt.5k | cut -f 10 -d "," | sort | uniq -c

3749  <=50K
1251  >50K


In [32]:
print(f'%>50K: {(1251/5000)*100}%')

%>50K: 25.019999999999996%


In [26]:
# Dev data
# Target categories and their counts
!cat hw1-data/income.dev.txt | cut -f 10 -d "," | sort | uniq -c

 764  <=50K
 236  >50K


In [31]:
print(f'%>50K: {(236/1000)*100}%')

%>50K: 23.599999999999998%


### What are the youngest and oldest ages in the training set? What are the least and most amounts of hours per week do people in this set work?

In [93]:
%%bash
# Youngest and oldeest ages in training set
echo "Youngest: $(cat hw1-data/income.train.txt.5k | cut -f 1 -d "," | sort | head -1)"  
echo "Oldest: $(cat hw1-data/income.train.txt.5k | cut -f 1 -d "," | sort | tail -1)" 

Youngest: 17
Oldest: 90


In [92]:
%%bash
# Least and most amount of working hours in training set
echo "Least num of working hours: $(cat hw1-data/income.train.txt.5k | cut -f 8 -d "," | sort | head -1)"  
echo "Highest num of working hours: $(cat hw1-data/income.train.txt.5k | cut -f 8 -d "," | sort | tail -1)" 

Least num of working hours:  1
Highest num of working hours:  99


### Why do we need to binarize all categorical fields?

<br>
Although there are some algorithms that could work on label data directly, most require the input data to be in numerical form. Meaning, as vectors in N dimensional feature space – so that they can apply vector ops as per the specified optimization algorithm to achieve an objective (like learning a pattern by minimizing a metric).

Okay, but can't we directly map categorical variables to numeric labels and represent them as vectors without binarizing them? Yes, but if the categorical variable is not an ordinal (but nominal), we would be introducing a bias by labeling some classes with bigger numbers and the others with smaller. How is that problematic? **We expect nominal classes will all have equal weightage (equidistant in ND space)** but they won't be. For example, if we map `["White", "Black", "Asian"]` to `[1, 2, 3]` repectively; each category (vector 1, 2, 3) has a different distance (1, 2, & 3 respectively). To nullify this bias, we use one hot encoding: `["White", "Black", "Asian"]` to `[[1, 0, 0], [0, 1, 0], [0, 0, 1]]` repectively. If you calculate distances now, they all are equidistant (as it should be). 

Binarizing categorical fields enables us to represent them **independently** in N dimensional space.
After one hot encoding, each possible category will have its own dimension and will not spill into other values with-in the same category OR other categories. 

This sort of representation will help modelling algorithms see the data in purest form without any induced language related bias. 

### If we do not count age and hours, what’s maximum possible Euclidean and Manhattan distances between two training examples? Explain.

In [90]:
%%bash

# Num of different categorical variables for each column:
for i in {1..10}
do
    echo "coln:($i) -> $(cat hw1-data/income.train.txt.5k| cut -f $i -d ","| sort | uniq | wc -l)"
done

coln:(1) ->       67
coln:(2) ->        7
coln:(3) ->       16
coln:(4) ->        7
coln:(5) ->       14
coln:(6) ->        5
coln:(7) ->        2
coln:(8) ->       73
coln:(9) ->       39
coln:(10) ->        2


The total number of all possible categories: `7 + 16 + 7 + 14 + 5 + 2 + 39 + 2 = 92`. 
So, each instance in training set is a vector of length 92 (when age and hours are omitted).

Theoritically, if X, Y are two training examples, the both distances will be maximized when `X = [1, 1, .... 1]; Y = [0, 0, .... 0]`.

In such case, both Euclidean distance is bounded by $d_{eu} = \sqrt{92}$ and Manhattan distances is bounded by $d_{mn} = 92$ 

This is because, each term in either formula (${x_i}^2 - {y_i}^2$ or $|x_i - y_i|$) are bounded by $1$ and there are 92 such terms, one for each possible category across all columns. 

### Why we do not want to binarize the two numerical fields, age and hours? What if we did? How should we define the distances on these two dimensions so that each field has equal weight? (In other words, the distance induced by each field should be bounded by 2 (N.B.: not 1! why?)).

[NO CLUE!]

### How many features do you have in total (i.e., the dimensionality)? Hint: should be around 100 90. How many features do you allocate for each of the 9 fields?


<br>
Keep age and hours continuos, we have 92.

Distribution:
```
coln:(1) ->        1
coln:(2) ->        7
coln:(3) ->       16
coln:(4) ->        7
coln:(5) ->       14
coln:(6) ->        5
coln:(7) ->        2
coln:(8) ->        1
coln:(9) ->       39
```

###  How many features would you have in total if you binarize all fields?

$90 + 67 + 73 = 230$

##  Calculating Manhattan and Euclidean Distances

In [108]:
import numpy as np
import pathlib

In [212]:
DATA_DIR = pathlib.Path("./hw1-data")

train_path = DATA_DIR / "income.train.txt.5k"
eval_path = DATA_DIR / "income.dev.txt"
test_path = DATA_DIR / "income.test.blind"
train_and_eval_path = DATA_DIR / "income.combined.6k"

COL_NAMES = [
    "age",
    "sector",
    "education",
    "marital-status",
    "occupation",
    "race",
    "gender",
    "hours-per-week",
    "country-of-origin",
    "target"
]

In [156]:
def txt_file_to_df(file_path):
        
    DELIMITER = ","
    
    def parse_row(row, delimiter=DELIMITER):
        cells = row.split(delimiter)
        parsed_row = [
            int(cell.strip()) 
            if cell.isnumeric() 
            else cell.strip()
            for cell in cells 
        ]
        
        return parsed_row
        
    data = []
    with open(file_path) as in_:
        raw_rows = in_.readlines()
        
    for row in raw_rows:
        parsed_row = parse_row(row)
        data.append(parsed_row)
    
    df = {col: val for col, val in zip(*[COL_NAMES, zip(*data)])}
    return df, data

In [157]:
# First person in eval
df, data = txt_file_to_df(train_path)
data[0], df.keys()

([50,
  'Self-emp-not-inc',
  'Bachelors',
  'Married-civ-spouse',
  'Exec-managerial',
  'White',
  'Male',
  '13',
  'United-States',
  '<=50K'],
 dict_keys(['age', 'sector', 'education', 'marital-status', 'occupation', 'race', 'gender', 'hours-per-week', 'country-of-origin', 'target']))

In [308]:
# One hot encoder

class OneHotEncoder(object):
    
    def __init__(self):
        pass
        
    def fit(self, column):
        self.unique_categories = list(set(column))
        self.catg_index = {catg: cid for cid, catg in enumerate(self.unique_categories)}
        
    def transform(self, column):
        ct, ck = 0, []
        self.ohe_column = np.zeros((len(column), len(self.unique_categories)), dtype=np.float64)
        for i, catg in enumerate(column):
            try:
                j = self.catg_index[catg]
                self.ohe_column[i, j] = 1
            except KeyError:
                ct += 1
                ck.append(catg)
                
        print(f'Failed {ct} times because of {ck}')
        return self.ohe_column
    
    def fit_transform(self, column):
        self.fit(column=column)
        
        return self.transform(column=column)

In [353]:
# One hot encoding categorical vars

def make_encoders(categorical_columns, train_df):
    '''
    Takes categorical column names and df to make encoder objects using data
    '''

    catg_columns = categorical_columns.keys()

    # Init encoder objects
    col_encoders = {col: OneHotEncoder() for col in catg_columns}
    
    # Train encoders
    for col in catg_columns:
        col_values = col_encoders[col].fit(train_df[col]) 
        
    return col_encoders


cols_catg = {
    "sector": 7,
    "education": 16,
    "marital-status": 4,
    "occupation": 14,
    "race": 5,
    "gender": 2,
    "country-of-origin": 39,
}

combined_df, _ = txt_file_to_df(train_and_eval_path)

make_encoders(categorical_columns=cols_catg, train_df=combined_df)

{'sector': <__main__.OneHotEncoder at 0x112bfedd0>,
 'education': <__main__.OneHotEncoder at 0x112bfe4d0>,
 'marital-status': <__main__.OneHotEncoder at 0x112bfee50>,
 'occupation': <__main__.OneHotEncoder at 0x11657d390>,
 'race': <__main__.OneHotEncoder at 0x11657d310>,
 'gender': <__main__.OneHotEncoder at 0x11657d4d0>,
 'country-of-origin': <__main__.OneHotEncoder at 0x11657dad0>}

In [354]:
'''
Takes file name and trained encoders as input to 
load, parse and transform dataset into modelling ready dataframe
'''

def txt_to_encoded_df(file_path, encoders):
    
    # Load and parse data
    df, _ = txt_file_to_df(file_path)
    
    # Encode and transform each col
    encoded_df = []
    for col in df.keys():

        if col == "target":
            continue

        elif col in encoders:
            encoder = encoders[col]
            col_values = encoder.transform(df[col]) 

        else:
            col_values = np.array(df[col], dtype=np.float64).reshape(len(df[col]), -1) / 50.

        encoded_df.append(col_values)

    # Make a flat dataset from all cols
    encoded_df = np.hstack(encoded_df)
    
    return encoded_df

In [359]:
# Example of this dataset's ETL cycle

combined_df_raw, _ = txt_file_to_df(train_and_eval_path)
one_hot_encoders = make_encoders(categorical_columns=cols_catg, train_df=combined_df_raw)
train_df = txt_to_encoded_df(file_path=train_path, encoders=one_hot_encoders)

print(train_df.shape)
train_df

Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
(5000, 93)


array([[1.  , 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.76, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [1.06, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [1.22, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.84, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.42, 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [360]:
train_df = txt_to_encoded_df(file_path=train_path, encoders=one_hot_encoders)
eval_df = txt_to_encoded_df(file_path=eval_path, encoders=one_hot_encoders)

Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []
Failed 0 times because of []


In [361]:
def euclidean_dist_bw_(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))

def manhattan_dist_bw_(v1, v2):
    return np.sum(np.abs(v1 - v2))

def retrive_closest_n(dists, n=3):
    return sorted(dists, key=lambda k: k[1])[:n]

##### Implementation test: 
`first_dev_person`s top 3 matches should be 4873, 4788 & 2592 from `train`

In [362]:
first_dev_person = eval_df[0]
eu_dists = pair_wise_dists(vector=first_dev_person, df=train_df, measure="euclidean")
mn_dists = pair_wise_dists(vector=first_dev_person, df=train_df, measure="man")

print(retrive_closest_n(eu_dists))
print(retrive_closest_n(mn_dists))

[[4872, 0.24738633753705963], [4787, 1.4147791347061915], [2591, 1.4156270695349111]]
[[4872, 0.30000000000000004], [4787, 2.04], [2591, 2.08]]


### Find the five (5) people closest to the last person (in Euclidean distance) in dev, and report their distances.

In [ ]:
def pair_wise_dists(vector, df, measure="euclidean"):
    
    all_dists = []
    for pid, pair_vector in enumerate(df):
        
        if measure == "euclidean":
            dist = euclidean_dist_bw_(vector, pair_vector)
        else:
            dist = manhattan_dist_bw_(vector, pair_vector)    
            
        all_dists.append([pid, dist])
        
    return all_dists

last_dev_person = eval_df[-1]

In [364]:
eu_dists = pair_wise_dists(vector=last_dev_person, df=train_df, measure="euclidean")
retrive_closest_n(eu_dists, n=5)

[[1010, 0.05999999999999983],
 [1713, 0.16000000000000014],
 [3769, 0.26],
 [2003, 0.2828427124746192],
 [2450, 0.3400000000000001]]

### Redo the above using Manhattan distance.

In [365]:
mn_dists = pair_wise_dists(vector=last_dev_person, df=train_df, measure="man")
retrive_closest_n(mn_dists, n=5)

[[1010, 0.05999999999999983],
 [1713, 0.16000000000000014],
 [3769, 0.26],
 [2450, 0.3400000000000001],
 [2003, 0.40000000000000024]]

### What are the 5-NN predictions for this person (Euclidean and Manhattan)? Are these predictions correct?

In [375]:
%%bash

echo "Ground truth: $(sed -n 1p hw1-data/income.dev.txt | cut -f 10 -d ',')"

echo "1011th line: $(sed -n 1011p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "1714th line: $(sed -n 1714p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "2004th line: $(sed -n 2004p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "2451th line: $(sed -n 2451p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "3770th line: $(sed -n 3770p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"

Ground truth:  <=50K
1011th line:  <=50K
1714th line:  <=50K
2004th line:  <=50K
2451th line:  <=50K
3770th line:  <=50K


The predictions are correct for 5-NN. 

### Redo all the above using 9-NN (i.e., find top-9 people closest to this person first).

In [376]:
eu_dists = pair_wise_dists(vector=last_dev_person, df=train_df, measure="euclidean")
retrive_closest_n(eu_dists, n=9)

[[1010, 0.05999999999999983],
 [1713, 0.16000000000000014],
 [3769, 0.26],
 [2003, 0.2828427124746192],
 [2450, 0.3400000000000001],
 [3698, 0.4004996878900156],
 [3680, 0.4386342439892262],
 [681, 0.5599999999999999],
 [2731, 1.4142135623730951]]

In [377]:
mn_dists = pair_wise_dists(vector=last_dev_person, df=train_df, measure="man")
retrive_closest_n(mn_dists, n=9)

[[1010, 0.05999999999999983],
 [1713, 0.16000000000000014],
 [3769, 0.26],
 [2450, 0.3400000000000001],
 [2003, 0.40000000000000024],
 [3698, 0.41999999999999993],
 [681, 0.5599999999999999],
 [3680, 0.58],
 [2731, 2.0]]

In [378]:
%%bash

echo "Ground truth: $(sed -n 1p hw1-data/income.dev.txt | cut -f 10 -d ',')"

echo "1011th line: $(sed -n 1011p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "1714th line: $(sed -n 1714p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "2004th line: $(sed -n 2004p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "2451th line: $(sed -n 2451p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "3770th line: $(sed -n 3770p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "2732th line: $(sed -n 2732p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "3681th line: $(sed -n 3681p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "682th line: $(sed -n 682p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"
echo "3699th line: $(sed -n 3699p hw1-data/income.train.txt.5k | cut -f 10 -d ',')"

Ground truth:  <=50K
1011th line:  <=50K
1714th line:  <=50K
2004th line:  <=50K
2451th line:  <=50K
3770th line:  <=50K
2732th line:  <=50K
3681th line:  <=50K
682th line:  <=50K
3699th line:  <=50K


The prediction is correct for 9-NN 